# Indus Valley Scripts - ICIT coded Text Analysis for Decipherment

# Multi-Line Texts Analysis

Dataset was created as a csv file from ICIT web site from raw html files for each for the Text
Data labels were changes and a linearized copy of the original text was added

icit_text_text_corpus.csv


!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install nltk
!pip install ipywidgets
!pip install -U dill
!pip3 install requests
!pip3 install -U spacy

In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import collections
import random
import traceback
import pickle

plt.style.use(style='seaborn')
%matplotlib inline

In [2]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from collections import defaultdict

In [3]:
drop_duplicate_texts = True

# Set the filters on data here
filter_by_site = False
filter_by_keywords = False
filter_by_text_length= False

site = 'Mohenjo-daro'
#site = 'Harappa'
#site = 'Dholavira'
#site = 'Rakhigarhi'
#keyword = "Bull"
#keyword = "Gaur"

min_text_length=1
max_text_length=50

num_rows_text_corpus= 4999

In [4]:
# Read the Text Corpus
orig_df=pd.read_csv('../../IndusCorpusUtils/data/icit_corpus/icit_text_text_corpus.csv',dtype=str, nrows=num_rows_text_corpus)
# set the max columns to none
pd.set_option('display.max_columns', None)


In [5]:
#Reverse text and add that as a new column
# Add text length as a column
list_reversed_text = []
for text in orig_df[orig_df.l_to_r_text!=''].l_to_r_text:
    # Tokenize to words
    # first split the string into chars
    chars = text.split(' ')
    length = len(chars)
    # then reverse the split string list and join with a space
    reversed_text = ' '.join(reversed(chars))
    list_reversed_text.append(reversed_text)
    
orig_df['reversed_text']= list_reversed_text #same as r_to_l text
orig_df['text_length']= orig_df['l_to_r_text'].str.len().div(3).round()

print("Dataframe has ", len(orig_df.index), " rows")

orig_df.head()

Dataframe has  4999  rows


,icit_id,site,keywords,text_class,lines,direction,text,signs,complete,alignment,sign height,text_images,linearized_text,l_to_r_text,r_to_l_text,reversed_text,text_length
0,1,Alamgirpur,NaN,SS,1,L/R,+410-017+,2,Y,Unordered,Unequal,NaN,410 017,410 017,017 410,017 410,2.0
1,2,Alamgirpur,NaN,SS,1,L/R,+410-017+,2,Y,NaN,NaN,NaN,410 017,410 017,017 410,017 410,2.0
2,3,Alamgirpur,NaN,SC,1,L/R,+405-017+,2,Y,NaN,NaN,NaN,405 017,405 017,017 405,017 405,2.0
3,4,Allahdino,NaN,??,1,NaN,+220-000+,1,N,NaN,NaN,NaN,220 000,000 220,000 220,220 000,2.0
4,5,Allahdino,Bull,UC,1,R/L,+740-235+,2,Y,NaN,NaN,NaN,740 235,235 740,740 235,740 235,2.0


In [6]:
def reverse_text(text):
    # first split the string into chars
    chars = text.split(' ')

    # then reverse the split string list and join with a space
    reversed_text = ' '.join(reversed(chars))
    return reversed_text

In [7]:
def reverse_as_list(a):
    list_reversed_text = []
    for text in a:
        # Tokenize to words
        # first split the string into chars
        chars = text.split(' ')
        length = len(chars)
        # then reverse the split string list and join with a space
        reversed_text = ' '.join(reversed(chars))
        list_reversed_text.append(reversed_text)
    return list_reversed_text


In [8]:
#Reverse text and add that as a new column
# Add text length as a column

def reverse_and_add_to_dataframe(a, this_df, reverse_text_column_name):
    list_reversed_text = []
    for text in a:
        # Tokenize to words
        # first split the string into chars
        chars = text.split(' ')
        length = len(chars)
        # then reverse the split string list and join with a space
        reversed_text = ' '.join(reversed(chars))
        list_reversed_text.append(reversed_text)

    this_df[reverse_text_column_name]= list_reversed_text 

In [9]:
df = orig_df

# Filter it ahead since we are using only Multiline df in this notebood
if(filter_by_site==True):
    #keep only the values that matches the provided site
    df = df[df['site'].str.contains(site) == True] 
    print("After filtering by site ", site, " it has ", len(df.index), " rows")

if(filter_by_keywords==True):
     #keep only the values that matches the provided keyword
    df = df[df['keywords'].str.contains(keyword) == True] 
    print("After filtering by keywords ", keyword, " it has ", len(df.index), " rows")

if(filter_by_text_length==True):
    df = df[(df['text_length'] > min_text_length) & (df['text_length']< max_text_length)]
    print("After filtering by text_length ",  " it has ", len(df.index), " rows")

# Multi Line Texts

In [10]:
#Keep the text that are multiline (has ''/'') in a separate dataframe

print("Multiline Text Analysis")
print("Dataframe has ", len(orig_df.index), " rows")

# Retain texts that are only wanted

#remove the values where the text is unclear
df = df[df['l_to_r_text'].str.contains('000') == False] 

print("After removing unclear texts, we have ", len(df.index), " rows")

if(drop_duplicate_texts):
    #Remove out duplicate inplace
    
    #df = df.drop_duplicates(subset ="text",
    #                     keep = 'first', inplace = False)
    
    # We will consider a text duplicate only of the keywords(pics) are different
    # In that case we will retain the first occurance of it
    df = df.drop_duplicates(subset =["text", "keywords"], inplace = False, keep = "first")
    print("After removing duplicate texts, we have ", len(df.index), " rows")

#Single sign ones don't have direction and won't have / and we need to explicitly include it
#Btw standardized_text is Left to right as in English
#df = df[(df['direction'].str.contains('/') == True) | (df['text_length'] ==1)] 

#print("After keeping only text with known direction, we have ", len(df.index), " rows")

#keep only the values that has multi-line text in a separate df
df_multi_line = df[df['text'].str.contains('/') == True] 


print("We have", len(df_multi_line.index), " rows of multi line texts")
print(df_multi_line.text)


Multiline Text Analysis
Dataframe has  4999  rows
After removing unclear texts, we have  3945  rows
After removing duplicate texts, we have  2793  rows
We have 78  rows of multi line texts
69                      +032-031/151-740-240-235+
70                  +032-031/740-222-235-002-861+
71              +032-031/850-032-530-740-741-456+
72                          +032-031/740-791-713+
74                              +032/226-032-817+
                          ...                    
4386                    +621/090-740-231-560-534+
4402                +790/740-100-415-740-257-840+
4705                        +740-900-003/741-002+
4729                                    +840/790+
4752    +605-740-142-067/002-374-310-350-495-834+
Name: text, Length: 78, dtype: object


In [11]:
# Multi-Line texts needs to be
# converted to separate texts. 
# There are a max of three parts, not all texts have three parts

text_part_1_list = []
text_part_2_list= []
text_part_3_list= []

direction_list = list(df_multi_line[df_multi_line.direction!=''].direction)

print(direction_list)
i=0

for text in df_multi_line[df_multi_line.text!=''].text :
    
   
    text = text.replace("+", "")
    text = text.replace("-", " ")
    text = text.replace("]", "")
    text = text.replace("[", "")
    
    #print("text:", text)
    # Tokenize to words
    # first split the string into chars
    parts = text.split("/")
    num_of_lines = len(parts)
    
    #print("parts:", parts)
    
    
    # TBD: Check noted direction and based on it make it all L to R.
    
    if(direction_list[i]=="R/L"):
        part_1 = reverse_text(parts[num_of_lines-1])
        part_2 = reverse_text(parts[num_of_lines-2])
        part_3 = reverse_text(parts[num_of_lines-3])
    else:
        part_1 = parts[num_of_lines-1]
        part_2 = parts[num_of_lines-2]
        part_3 = parts[num_of_lines-3]
        
    
    text_part_1_list.append(part_1)
    text_part_2_list.append(part_2)
    if(num_of_lines==3):
        print("--------", text, "----------")
        text_part_3_list.append(part_3)
    else: text_part_3_list.append(" ")
        
    
df_multi_line['text_part_1']= text_part_1_list 
df_multi_line['text_part_2']= text_part_2_list
df_multi_line['text_part_3']= text_part_3_list

# Add reverse of part_1 and part_2 as separate columns
a =df_multi_line[df_multi_line.text_part_1!=''].text_part_1
reverse_and_add_to_dataframe(a, df_multi_line, 'text_part_1_rev')

a =df_multi_line[df_multi_line.text_part_2!=''].text_part_2
reverse_and_add_to_dataframe(a, df_multi_line, 'text_part_2_rev')

a =df_multi_line[df_multi_line.text_part_3!=''].text_part_3
reverse_and_add_to_dataframe(a, df_multi_line, 'text_part_3_rev')

df_multi_line.to_csv('multi_line_texts.csv')


df_multi_line.head()

['R/L', 'R/L', 'R/L', 'R/L', 'L/R', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', nan, 'R/L', 'R/L', nan, 'R/L', 'R/L', nan, nan, 'R/L', nan, 'R/L', 'R/L', 'R/L', 'R/L', 'BUS', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'BUS', 'R/L', 'R/L', nan, 'T/B', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', nan, 'R/L', 'R/L', nan, 'R/L', 'R/L', 'R/L', 'R/L', 'R/L', nan, 'R/L', 'R/L', nan, 'BUS']
-------- 159 097/520 033 705/740 717 220 415 806 ----------
-------- 832/006 740/090 ----------
-------- 953 817 390 125 308 809 033/740 760 706 466/520 231 240 235 002 820 ----------


/var/folders/0n/5lj8g_xs6xngc99x77p4k50h0000gp/T/ipykernel_51798/401783368.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_multi_line['text_part_1']= text_part_1_list
/var/folders/0n/5lj8g_xs6xngc99x77p4k50h0000gp/T/ipykernel_51798/401783368.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_multi_line['text_part_2']= text_part_2_list
/var/folders/0n/5lj8g_xs6xngc99x77p4k50h0000gp/T/ipykernel_51798/401783368.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,icit_id,site,keywords,text_class,lines,direction,text,signs,complete,alignment,sign height,text_images,linearized_text,l_to_r_text,r_to_l_text,reversed_text,text_length,text_part_1,text_part_2,text_part_3,text_part_1_rev,text_part_2_rev,text_part_3_rev
69,70,Chanhujo-daro,Bull1:II,2L,2,R/L,+032-031/151-740-240-235+,6,Y,Linear,Unequal,NaN,032 031 151 740 240 235,235 240 740 151 031 032,032 031 151 740 240 235,032 031 151 740 240 235,8.0,235 240 740 151,031 032,,151 740 240 235,032 031,
70,71,Chanhujo-daro,Bull1,2L,2,R/L,+032-031/740-222-235-002-861+,7,Y,Unordered,Unequal,NaN,032 031 740 222 235 002 861,861 002 235 222 740 031 032,032 031 740 222 235 002 861,032 031 740 222 235 002 861,9.0,861 002 235 222 740,031 032,,740 222 235 002 861,032 031,
71,72,Chanhujo-daro,Gaur,2L,2,R/L,+032-031/850-032-530-740-741-456+,8,Y,Unordered,Unequal,NaN,032 031 850 032 530 740 741 456,456 741 740 530 032 850 031 032,032 031 850 032 530 740 741 456,032 031 850 032 530 740 741 456,10.0,456 741 740 530 032 850,031 032,,850 032 530 740 741 456,032 031,
72,73,Chanhujo-daro,Bull1,2L,2,R/L,+032-031/740-791-713+,5,Y,Unordered,Unequal,NaN,032 031 740 791 713,713 791 740 031 032,032 031 740 791 713,032 031 740 791 713,6.0,713 791 740,031 032,,740 791 713,032 031,
74,75,Chanhujo-daro,NaN,2L,2,L/R,+032/226-032-817+,4,Y,Linear,Equal,NaN,032 226 032 817,032 226 032 817,817 032 226 032,817 032 226 032,5.0,817 032 226,032,,226 032 817,032,


In [12]:
df_multi_line_r_to_l = df_multi_line[df_multi_line['direction'].str.contains('R/L')== True]
print(len(df_multi_line_r_to_l.index))

df_multi_line_l_to_r = df_multi_line[df_multi_line['direction'].str.contains('L/R')== True]
print(len(df_multi_line_l_to_r.index))

df_multi_line_no_dir = df_multi_line[df_multi_line['direction'].str.contains('/')== False]
print(len(df_multi_line_no_dir.index))

63
1
3


### n-gram Models

In [13]:
from nltk.lm.preprocessing import pad_both_ends
from nltk.util import everygrams
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.lm.models import MLE
from nltk.lm.models import KneserNeyInterpolated

In [14]:
seed=8

In [15]:
# Test-load that was created in Language model from disk
model_quad_kn = pickle.load(open('kn_quad_model.pkl', 'rb'))
model_quad_kn_rev = pickle.load(open('kn_rev_quad_model.pkl', 'rb'))

#test
print("**count of 390:", model_quad_kn.counts['390'])
print("**count of 390 rev:", model_quad_kn_rev.counts['390'])

**count of 390: 158
**count of 390 rev: 158


In [16]:
def get_perplexity(model1, k, this_texts, this_tokenized_text,seed=8):
    perplexity_list = []
    try:
        this_data_list = []
        this_data_list, _= padded_everygram_pipeline(k, this_tokenized_text)
        for i, test in enumerate(this_data_list):
            if(this_texts[i]!=" "): 
                perp = model1.perplexity(test)
                if(perp>100000): perp = 100000
            else: perp = 0
            perplexity_list.append(perp)
            #print("Perplexity( {0}):{1}".format(this_texts[i], model1.perplexity(test)))
    except Exception as e:
                print("Exception:", e.__class__, "get_perplexity")
                traceback.print_exc()
    return this_texts, perplexity_list

In [17]:
# Note: fwd: l/r, rev: r/l
def get_text_direction0(model_fwd, k, text_list,tokenized_text_fwd,tokenized_text_rev,direction_list, seed):
    
    match, either,total, lr_count =0,0,0,0
    text_list, perplexity_list_fwd_text = get_perplexity(model_fwd, k,text_list,tokenized_text_fwd,seed)
    text_list_rev, perplexity_list_rev_text = get_perplexity(model_fwd, k,text_list,tokenized_text_rev,seed)
  
    sub_list_text = [x1 - x2 for (x1, x2) in zip(perplexity_list_fwd_text , perplexity_list_rev_text )]
    
    # Intuitively, if a model assigns a high probability to the test set, it means that it is not surprised to see it (it’s not perplexed by it), which means that it has a good understanding of how the language works.
    # If the text does better (lower perplexity) with L to R (fwd) model, than with R to L (rev) model, than the text is L to R
    # If the text does better (lower perplexity) with R to L (rev) model, than with L to R (fwd) model, than the text is R to L
    for i, num in enumerate(sub_list_text):
        print("(Perp of fwd - Perp of rev)", (perplexity_list_fwd_text[i],perplexity_list_rev_text[i]))
        if(num>0):
            ans = "L/R"
            print(text_list[i], ": L/R ------------->>>", "(",num, ")")
            lr_count= lr_count+1
        elif(num==0): 
            ans ="Either"
            print(text_list[i], ": Equal direction", "(",num, ")")
        else: 
            ans= "R/L"
            print(text_list[i], ": R/L " ,"(",num, ")")
        
        total= total+1
        if(len(direction_list)!=0):
            if(ans==direction_list[i] ):
                match = match +1
                print("Match")
            elif(ans=="Either"):
                print("Either")
                either= either +1
            else: print("******** No Match *******")
    
    print()
    
    if(len(direction_list)!=0):
        print("Matched:", match, "out of total", total, "%", (match*100/total), "Either:", either)
    else:
        print("Completed", total, "texts", "l/r texts:", lr_count)
    return sub_list_text

# Test-0: Direction check for Manual Test

In [27]:
k=4
this_tokenized_text_list = []
this_tokenized_text_list_rev =[]
direction_list= []

#text = "861 002 705 255 740 090"
text = "235 240 740"
text_rev = reverse_text(text)

this_tokenized_text= word_tokenize(text)
this_tokenized_text_list.append(this_tokenized_text)
print(this_tokenized_text_list)

this_tokenized_text_rev= word_tokenize(text_rev)
this_tokenized_text_list_rev.append(this_tokenized_text_rev)
print(this_tokenized_text_list_rev)

perp_diff_test_text_list = get_text_direction0(model_quad_kn,k,text,this_tokenized_text_list,this_tokenized_text_list_rev,direction_list, seed)


[['235', '240', '740']]
[['740', '240', '235']]
(Perp of fwd - Perp of rev) (34.96755387359558, 88.30643047833453)
2 : R/L  ( -53.33887660473896 )

Completed 1 texts l/r texts: 0


# Test-1: Direction check for Train Set

In [19]:
# Use Train set and get direction and find out how many are predicted correctly
# Read the signs
df_train_texts=pd.read_csv('train_texts.csv',dtype=str)
# set the max columns to none
pd.set_option('display.max_columns', None)
df_train_texts.head()

,Unnamed: 0,l_to_r_text,direction,site
0,0,806 740 045,R/L,Mohenjo-daro
1,1,455 590 002 005 368,R/L,Mohenjo-daro
2,2,365 527,L/R,Wattoowala
3,3,327 740 090,R/L,Harappa
4,4,140 920 944 002 240 482 740,R/L,Mohenjo-daro


In [20]:
k=4
seed=8

train_text_list = list(df_train_texts[df_train_texts.l_to_r_text!=''].l_to_r_text)
tokenized_train_text_fwd_list = list(df_train_texts[df_train_texts.l_to_r_text!=''].l_to_r_text.apply(word_tokenize))

#print(tokenized_train_text_fwd_list)
list_reversed_text = []
for text in df_train_texts[df_train_texts.l_to_r_text!=''].l_to_r_text:
    # Tokenize to words
    # first split the string into chars
    chars = text.split(' ')
    length = len(chars)
    # then reverse the split string list and join with a space
    reversed_text = ' '.join(reversed(chars))
    list_reversed_text.append(reversed_text)

df_train_texts['r_to_l_text']= list_reversed_text

#print(df_train_texts)

train_text_rev_list = list(df_train_texts.r_to_l_text)
tokenized_train_text_rev_list = list(df_train_texts[df_train_texts.r_to_l_text!=''].r_to_l_text.apply(word_tokenize))
direction_list = list(df_train_texts[df_train_texts.direction!=''].direction)

print("train_text_l:")

# This takes too Long. So uncomment it when needed
#perp_diff_test_text_list = get_text_direction0(model_quad_kn,k,train_text_list,tokenized_train_text_fwd_list,tokenized_train_text_rev_list,direction_list, seed)

print("Over!")

train_text_l:
Over!


## Test-2: Direction check for Test Set

In [21]:
# Use Test set and get direction and find out how many are predicted correctly
# Read the signs
df_test_texts=pd.read_csv('test_texts.csv',dtype=str)
# set the max columns to none
pd.set_option('display.max_columns', None)
df_test_texts.head()

,Unnamed: 0,l_to_r_text,direction,site
0,0,002 861 096,L/R,Mohenjo-daro
1,1,491 817 740,R/L,Harappa
2,2,933 002 004 405,R/L,Harappa
3,3,924 001 319 031 055 002 150 416,R/L,Susa
4,4,322 920 740 001 003 137,R/L,Mohenjo-daro


In [22]:
k=4
seed=8

test_text_list = list(df_test_texts[df_test_texts.l_to_r_text!=''].l_to_r_text)
tokenized_test_text_fwd_list = list(df_test_texts[df_test_texts.l_to_r_text!=''].l_to_r_text.apply(word_tokenize))

a = df_test_texts[df_test_texts.l_to_r_text!=''].l_to_r_text
list_reversed_text = reverse_as_list(a)

df_test_texts['r_to_l_text']= list_reversed_text

test_text_rev_list = list(df_test_texts[df_test_texts.r_to_l_text!=''].r_to_l_text)
tokenized_test_text_rev_list = list(df_test_texts[df_test_texts.r_to_l_text!=''].r_to_l_text.apply(word_tokenize))
direction_list = list(df_test_texts[df_test_texts.direction!=''].direction)

print("test_text_l:")
perp_diff_test_text_list = get_text_direction0(model_quad_kn,k,test_text_list,tokenized_test_text_fwd_list,tokenized_test_text_rev_list,direction_list, seed)


test_text_l:
(Perp of fwd - Perp of rev) (227.6227045505217, 72.38637689954308)
002 861 096 : L/R ------------->>> ( 155.2363276509786 )
Match
(Perp of fwd - Perp of rev) (23.531253810912684, 224.94851211848277)
491 817 740 : R/L  ( -201.41725830757008 )
Match
(Perp of fwd - Perp of rev) (100000, 100000)
933 002 004 405 : Equal direction ( 0 )
Either
(Perp of fwd - Perp of rev) (405.18670874252126, 368.8781641675571)
924 001 319 031 055 002 150 416 : L/R ------------->>> ( 36.30854457496417 )
******** No Match *******
(Perp of fwd - Perp of rev) (101.67601895723838, 324.4710220475624)
322 920 740 001 003 137 : R/L  ( -222.79500309032403 )
Match
(Perp of fwd - Perp of rev) (42.75889978444995, 127.18414350799125)
540 002 806 240 017 575 740 679 : R/L  ( -84.4252437235413 )
Match
(Perp of fwd - Perp of rev) (181.6007950779444, 38.57172828698213)
156 176 741 060 920 : L/R ------------->>> ( 143.02906679096228 )
Match
(Perp of fwd - Perp of rev) (191.47162522095704, 606.6979251493755)
720 4

## Test-2: Find Direction for Multi-Part texts

In [23]:
k=4
seed=8
text_list, perplexity_list = None, None
direction_list = []
    
#Perplexity calculations for Multi-Line texts
#use a model to check the perplexity of test texts

text_part_1_list_fwd = list(df_multi_line[df_multi_line.text_part_1!=''].text_part_1)
tokenized_text_part_1_fwd = list(df_multi_line[df_multi_line.text_part_1!=''].text_part_1.apply(word_tokenize))

text_part_2_list_fwd = list(df_multi_line[df_multi_line.text_part_2!=''].text_part_2)
tokenized_text_part_2_fwd = list(df_multi_line[df_multi_line.text_part_2!=''].text_part_2.apply(word_tokenize))

text_part_3_list_fwd = list(df_multi_line[df_multi_line.text_part_3!=''].text_part_3)
tokenized_text_part_3_fwd = list(df_multi_line[df_multi_line.text_part_3!=''].text_part_3.apply(word_tokenize))

text_part_1_list_rev = list(df_multi_line[df_multi_line.text_part_1_rev!=''].text_part_1_rev)
tokenized_text_part_1_rev = list(df_multi_line[df_multi_line.text_part_1_rev!=''].text_part_1_rev.apply(word_tokenize))

text_part_2_list_rev = list(df_multi_line[df_multi_line.text_part_2_rev!=''].text_part_2_rev)
tokenized_text_part_2_rev = list(df_multi_line[df_multi_line.text_part_2_rev!=''].text_part_2_rev.apply(word_tokenize))

text_part_3_list_rev = list(df_multi_line[df_multi_line.text_part_3_rev!=''].text_part_3_rev)
tokenized_text_part_3_rev = list(df_multi_line[df_multi_line.text_part_3_rev!=''].text_part_3_rev.apply(word_tokenize))


print("text-part-1:")
perp_diff_text_part1_list = get_text_direction0(model_quad_kn,k,text_part_1_list,tokenized_text_part_1_fwd, tokenized_text_part_1_rev,direction_list,seed)

print("text-part-2:")
perp_diff_text_part2_list = get_text_direction0(model_quad_kn,k,text_part_2_list,tokenized_text_part_2_fwd,tokenized_text_part_2_rev,direction_list,seed)

print("text-part-3:")
perp_diff_text_part3_list = get_text_direction0(model_quad_kn,k,text_part_3_list,tokenized_text_part_3_fwd,tokenized_text_part_3_rev,direction_list,seed)



text-part-1:
(Perp of fwd - Perp of rev) (35.869535460602684, 123.81983085432947)
235 240 740 151 : R/L  ( -87.95029539372678 )
(Perp of fwd - Perp of rev) (21.549845226028197, 149.33012534300403)
861 002 235 222 740 : R/L  ( -127.78028011697583 )
(Perp of fwd - Perp of rev) (204.58085622442007, 257.80320695139653)
456 741 740 530 032 850 : R/L  ( -53.22235072697646 )
(Perp of fwd - Perp of rev) (100000, 100000)
713 791 740 : Equal direction ( 0 )
(Perp of fwd - Perp of rev) (64.10890779652873, 135.6646038842344)
817 032 226 : R/L  ( -71.55569608770567 )
(Perp of fwd - Perp of rev) (31.759845533401027, 56.916494012895306)
817 002 : R/L  ( -25.15664847949428 )
(Perp of fwd - Perp of rev) (28.82451834840455, 103.16586505464039)
055 740 : R/L  ( -74.34134670623584 )
(Perp of fwd - Perp of rev) (65.3649281753678, 48.23297676708312)
031 032 : L/R ------------->>> ( 17.13195140828467 )
(Perp of fwd - Perp of rev) (143.00766628198556, 191.44237619893696)
500 031 002 221 740 090 : R/L  ( -48.4

(Perp of fwd - Perp of rev) (65.3649281753678, 48.23297676708312)
031 032 : L/R ------------->>> ( 17.13195140828467 )
(Perp of fwd - Perp of rev) (65.3649281753678, 48.23297676708312)
031 032 : L/R ------------->>> ( 17.13195140828467 )
(Perp of fwd - Perp of rev) (65.3649281753678, 48.23297676708312)
031 032 : L/R ------------->>> ( 17.13195140828467 )
(Perp of fwd - Perp of rev) (65.3649281753678, 48.23297676708312)
031 032 : L/R ------------->>> ( 17.13195140828467 )
(Perp of fwd - Perp of rev) (38.343267509791964, 38.343267509791964)
032 : Equal direction ( 0.0 )
(Perp of fwd - Perp of rev) (23.67487654842427, 42.66101499351434)
240 636 740 : R/L  ( -18.986138445090074 )
(Perp of fwd - Perp of rev) (40.51211696353788, 40.51211696353788)
790 : Equal direction ( 0.0 )
(Perp of fwd - Perp of rev) (68.98361584634875, 101.39683033645942)
690 151 : R/L  ( -32.41321449011066 )
(Perp of fwd - Perp of rev) (178.38925667285704, 182.9274314991965)
521 521 817 : R/L  ( -4.538174826339457 )
(P

In [24]:
import numpy as np
text_perp_diff_parts_df = pd.DataFrame(np.column_stack([perp_diff_text_part1_list , perp_diff_text_part2_list , perp_diff_text_part3_list]), 
                               columns=['perp_diff_text_part1', 'perp_diff_text_part2', 'perp_diff_text_part3'])

text_perp_diff_parts_df.head()

,perp_diff_text_part1,perp_diff_text_part2,perp_diff_text_part3
0,-87.950295,17.131951,0.0
1,-127.780280,17.131951,0.0
2,-53.222351,17.131951,0.0
3,0.000000,17.131951,0.0
4,-71.555696,0.000000,0.0


In [25]:
# Find the perplexity of the model for different combination of
# how the three parts of text combine
# 

direction_list = []

df_multi_line['text_form_1'] = df_multi_line['text_part_2'] + " " + df_multi_line['text_part_1']
df_multi_line['text_form_2'] = df_multi_line['text_part_2_rev'] + " " + df_multi_line['text_part_1']
df_multi_line['text_form_3'] = df_multi_line['text_part_2'] + " " + df_multi_line['text_part_1_rev']
df_multi_line['text_form_4'] = df_multi_line['text_part_2_rev'] + " " + df_multi_line['text_part_1_rev']

df_multi_line['text_form_1_rev'] = reverse_as_list(df_multi_line['text_form_1'])
df_multi_line['text_form_2_rev'] = reverse_as_list(df_multi_line['text_form_2'])
df_multi_line['text_form_3_rev'] = reverse_as_list(df_multi_line['text_form_3'])
df_multi_line['text_form_4_rev'] = reverse_as_list(df_multi_line['text_form_4'])
    
text_form_1_list = list(df_multi_line['text_form_1'])
tokenized_text_form_1= list(df_multi_line[df_multi_line.text_form_1!=''].text_form_1.apply(word_tokenize))
text_form_1_list_rev = list(df_multi_line['text_form_1_rev'])
tokenized_text_form_1_rev = list(df_multi_line[df_multi_line.text_form_1_rev!=''].text_form_1_rev.apply(word_tokenize))


text_form_2_list = list(df_multi_line['text_form_2'])
tokenized_text_form_2= list(df_multi_line[df_multi_line.text_form_2!=''].text_form_2.apply(word_tokenize))
text_form_2_list_rev = list(df_multi_line['text_form_2_rev'])
tokenized_text_form_2_rev = list(df_multi_line[df_multi_line.text_form_2_rev!=''].text_form_2_rev.apply(word_tokenize))


text_form_3_list = list(df_multi_line['text_form_3'])
tokenized_text_form_3= list(df_multi_line[df_multi_line.text_form_3!=''].text_form_3.apply(word_tokenize))
text_form_3_list_rev = list(df_multi_line['text_form_3_rev'])
tokenized_text_form_3_rev = list(df_multi_line[df_multi_line.text_form_3_rev!=''].text_form_3_rev.apply(word_tokenize))


text_form_4_list = list(df_multi_line['text_form_4'])
tokenized_text_form_4= list(df_multi_line[df_multi_line.text_form_4!=''].text_form_4.apply(word_tokenize))
text_form_4_list_rev = list(df_multi_line['text_form_4_rev'])
tokenized_text_form_4_rev = list(df_multi_line[df_multi_line.text_form_4_rev!=''].text_form_4_rev.apply(word_tokenize))


print("text_form_1: text-part-2 + text-part-1:")
perp_diff_text_form_1_list = get_text_direction0(model_quad_kn,k,text_form_1_list,tokenized_text_form_1, tokenized_text_form_1_rev,direction_list, seed)

print("text_form_2: text-part-2_rev + rev text-part-1:")
perp_diff_text_form_2_list = get_text_direction0(model_quad_kn,k,text_form_2_list,tokenized_text_form_2, tokenized_text_form_2_rev,direction_list, seed)

print("text_form_3: text-part-2 + rev text-part-1_rev:")
perp_diff_text_form_3_list = get_text_direction0(model_quad_kn,k,text_form_3_list,tokenized_text_form_3, tokenized_text_form_3_rev,direction_list, seed)

print("text_form_4: text-part-2_rev + rev text-part-1-rev:")
perp_diff_text_form_4_list = get_text_direction0(model_quad_kn,k,text_form_4_list,tokenized_text_form_4, tokenized_text_form_4_rev,direction_list, seed)


text_form_1: text-part-2 + text-part-1:


/var/folders/0n/5lj8g_xs6xngc99x77p4k50h0000gp/T/ipykernel_51798/1848862498.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_multi_line['text_form_1'] = df_multi_line['text_part_2'] + " " + df_multi_line['text_part_1']
/var/folders/0n/5lj8g_xs6xngc99x77p4k50h0000gp/T/ipykernel_51798/1848862498.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_multi_line['text_form_2'] = df_multi_line['text_part_2_rev'] + " " + df_multi_line['text_part_1']
/var/folders/0n/5lj8g_xs6xngc99x77p4k50h0000gp/T/ipykernel

(Perp of fwd - Perp of rev) (73.0996070435067, 104.26093550178118)
031 032 235 240 740 151 : R/L  ( -31.161328458274483 )
(Perp of fwd - Perp of rev) (37.375819463423696, 198.7176513290551)
031 032 861 002 235 222 740 : R/L  ( -161.34183186563138 )
(Perp of fwd - Perp of rev) (256.5209930982231, 193.52069486304248)
031 032 456 741 740 530 032 850 : L/R ------------->>> ( 63.000298235180594 )
(Perp of fwd - Perp of rev) (100000, 100000)
031 032 713 791 740 : Equal direction ( 0 )
(Perp of fwd - Perp of rev) (90.32203047028725, 158.4788625557656)
032 817 032 226 : R/L  ( -68.15683208547836 )
(Perp of fwd - Perp of rev) (57.52063368515247, 83.41730600030299)
240 636 740 817 002 : R/L  ( -25.896672315150518 )
(Perp of fwd - Perp of rev) (57.10578371477245, 135.10285617442383)
790 055 740 : R/L  ( -77.99707245965138 )
(Perp of fwd - Perp of rev) (140.15616193597944, 115.09869266675776)
690 151 031 032 : L/R ------------->>> ( 25.057469269221684 )
(Perp of fwd - Perp of rev) (249.55911933027

(Perp of fwd - Perp of rev) (69.41806523408749, 142.80049514563578)
032 031 235 240 740 151 : R/L  ( -73.38242991154829 )
(Perp of fwd - Perp of rev) (36.34788220664688, 239.95289265794432)
032 031 861 002 235 222 740 : R/L  ( -203.60501045129743 )
(Perp of fwd - Perp of rev) (245.62421591403114, 230.1790511328873)
032 031 456 741 740 530 032 850 : L/R ------------->>> ( 15.44516478114383 )
(Perp of fwd - Perp of rev) (100000, 100000)
032 031 713 791 740 : Equal direction ( 0 )
(Perp of fwd - Perp of rev) (90.32203047028725, 158.4788625557656)
032 817 032 226 : R/L  ( -68.15683208547836 )
(Perp of fwd - Perp of rev) (57.22082988053402, 63.39386016277007)
740 636 240 817 002 : R/L  ( -6.1730302822360485 )
(Perp of fwd - Perp of rev) (57.10578371477245, 135.10285617442383)
790 055 740 : R/L  ( -77.99707245965138 )
(Perp of fwd - Perp of rev) (153.75755730887957, 104.86956161177311)
151 690 031 032 : L/R ------------->>> ( 48.88799569710646 )
(Perp of fwd - Perp of rev) (255.3685524491135

(Perp of fwd - Perp of rev) (151.89240785656517, 68.52295754952945)
031 032 151 740 240 235 : L/R ------------->>> ( 83.36945030703572 )
(Perp of fwd - Perp of rev) (157.7949569439949, 40.64132134348644)
031 032 740 222 235 002 861 : L/R ------------->>> ( 117.15363560050845 )
(Perp of fwd - Perp of rev) (258.2819511117298, 160.45119977051561)
031 032 850 032 530 740 741 456 : L/R ------------->>> ( 97.83075134121421 )
(Perp of fwd - Perp of rev) (100000, 100000)
031 032 740 791 713 : Equal direction ( 0 )
(Perp of fwd - Perp of rev) (108.11652835598197, 114.47470211691149)
032 226 032 817 : R/L  ( -6.3581737609295175 )
(Perp of fwd - Perp of rev) (54.04364823102363, 62.159711779067585)
240 636 740 002 817 : R/L  ( -8.116063548043954 )
(Perp of fwd - Perp of rev) (92.4041635335936, 49.17118381475828)
790 740 055 : L/R ------------->>> ( 43.232979718835324 )
(Perp of fwd - Perp of rev) (108.59765811217201, 136.7940183431041)
690 151 032 031 : R/L  ( -28.1963602309321 )
(Perp of fwd - Pe

(Perp of fwd - Perp of rev) (132.0761616682016, 84.24122002693373)
032 031 151 740 240 235 : L/R ------------->>> ( 47.83494164126786 )
(Perp of fwd - Perp of rev) (159.92912686313431, 44.923405475031316)
032 031 740 222 235 002 861 : L/R ------------->>> ( 115.005721388103 )
(Perp of fwd - Perp of rev) (211.755698201632, 184.58319965420475)
032 031 850 032 530 740 741 456 : L/R ------------->>> ( 27.17249854742724 )
(Perp of fwd - Perp of rev) (100000, 100000)
032 031 740 791 713 : Equal direction ( 0 )
(Perp of fwd - Perp of rev) (108.11652835598197, 114.47470211691149)
032 226 032 817 : R/L  ( -6.3581737609295175 )
(Perp of fwd - Perp of rev) (53.761966853106124, 25.90801014261875)
740 636 240 002 817 : L/R ------------->>> ( 27.853956710487374 )
(Perp of fwd - Perp of rev) (92.4041635335936, 49.17118381475828)
790 740 055 : L/R ------------->>> ( 43.232979718835324 )
(Perp of fwd - Perp of rev) (119.1364718478779, 88.54642009718147)
151 690 032 031 : L/R ------------->>> ( 30.59005

In [26]:
import numpy as np
text_perplexity_forms_df = pd.DataFrame(np.column_stack([perp_diff_text_form_1_list, perp_diff_text_form_2_list, perp_diff_text_form_3_list, perp_diff_text_form_4_list]), 
                               columns=['perp_diff_text_form_1', 'perp_diff_text_form_2', 'perp_diff_text_form_3', 'perp_diff_text_form_4'])

text_perplexity_forms_df.head()

,perp_diff_text_form_1,perp_diff_text_form_2,perp_diff_text_form_3,perp_diff_text_form_4
0,-31.161328,-73.382430,83.369450,47.834942
1,-161.341832,-203.605010,117.153636,115.005721
2,63.000298,15.445165,97.830751,27.172499
3,0.000000,0.000000,0.000000,0.000000
4,-68.156832,-68.156832,-6.358174,-6.358174
